In [ ]:
     #-데이터를 로딩하는 기능은 넣지 않는다 => 공용으로 사용하기 때문(메모리 낭비가 심하다)
    #사용하는 데이터를 field(변수로 선언)
    #사용하는 기능은 함수(method)로 표현
    #-tensorflow graph를 그리는 기능(모델을 구축)
    #-학습하는 기능
    #-정확도 측정
    #-예측작업

In [69]:
import pandas as pd
import numpy as np
import tensorflow as tf
tf.reset_default_graph()

#0. class 
class CNNModel:
    def __init__(self,name, x,y,sess):  #생성자(constructor)
        self.x = x
        self.y = y  
        self.sess = sess
        self.name = name
    
    def build_model(self):
        with tf.variable_scope(self.name):
            
            #2. placeholer
            column_x = self.x.shape[1]
            column_y = self.y.shape[1]
            self.X=tf.placeholder(shape=[None,column_x], dtype=tf.float32)
            self.Y=tf.placeholder(shape=[None,column_y], dtype=tf.float32)
            self.keep_rate = tf.placeholder(dtype=tf.float32)

            #3. Convolution Layer
            #Replace the image data with a squared array.
            X_img = tf.reshape(self.X, shape=[-1,28,28,1])
            W1=tf.Variable(tf.random_normal([3,3,1,32],stddev=0.01))
            L1 = tf.layers.conv2d(inputs = X_img, filters = 32, kernel_size = [3,3],padding = "SAME", strides = 1, activation = tf.nn.relu)
            
            #MaxPooling
            L1 = tf.layers.max_pooling2d(inputs=L1, pool_size = [2,2]
                                        ,padding = "SAME", strides = 2 )
            L1 = tf.layers.dropout(inputs=L1, rate=0.3)
            #Filter, Convolution, ReLU
            L2 = tf.layers.conv2d(inputs = L1, filters = 64, kernel_size = [3,3]
                                 ,padding = "SAME", strides = 1
                                  , activation = tf.nn.relu)

            #MaxPooling
            L2 = tf.layers.max_pooling2d(inputs = L2, pool_size = [2,2]
                                        ,padding = "SAME", strides = 2 )
            #4. FC(Neural Network)
            L2 = tf.layers.dropout(inputs=L2, rate=0.3)
            L2 = tf.reshape(L2, shape = [-1,7*7*64])

            #5. Weight & bias
            
#   dense1 = tf.layers.dense(inputs = L2, units = 64, activation = tf.nn.relu)
#   dense1 = tf.layers.dropout(inputs = dense1, rate = self.keep_prob)
# W와 b가 사라진다. 변수들의 이름 때문에 생겼던 문제가 사라진다. 
#self.H = tf.layers.dense(inputs = dense1,units = 10) 대신 H 새로 짜야한다. 
            W2 = tf.get_variable("weight2", shape= [7*7*64,256]
                                 ,initializer 
                                 = tf.contrib.layers.xavier_initializer())
            b2 = tf.Variable(tf.random_normal([256]), name ="bias2")

            _layer1 = tf.nn.relu(tf.matmul(L2,W2) + b2)
            layer1 = tf.nn.dropout(_layer1, keep_prob = self.keep_rate)

            W3 = tf.get_variable("weight3", shape = [256,256]
                                 ,initializer 
                                 = tf.contrib.layers.xavier_initializer())
            b3 = tf.Variable(tf.random_normal([256]), name = "bias3")

            _layer2 = tf.nn.relu(tf.matmul(layer1,W3) + b3)
            layer2 = tf.nn.dropout(_layer2,  keep_prob = self.keep_rate)

            W4 = tf.get_variable("weight4", shape = [256,10]
                                 ,initializer 
                                 = tf.contrib.layers.xavier_initializer())
            b4 = tf.Variable(tf.random_normal([10]), name = "bias4")

            #6. Hypothesis
            self.H = tf.matmul(layer2, W4) + b4 #데이터를 가져와서 

            #7. Cost
            self.cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2
                                       (logits = self.H, labels= self.Y))
            #8. Train
            self.train = tf.train.AdamOptimizer(learning_rate = 0.001).minimize(self.cost)
            predict = tf.argmax(self.H,1) # tf붙은 것은 빌드로 옮기기
            correct = tf.equal(predict,tf.argmax(self.Y,1))
            self.accuracy = tf.reduce_mean(tf.cast(correct, dtype = tf.float32))
#             self.sess.run(accuracy, feed_dict = {self.X:test_x, self.Y: test_y, self.keep_rate:1})
        
    def train_net(self,train_x, train_y, rate):
        _, cost_val = self.sess.run([self.train, self.cost]
                                    , feed_dict={self.X:train_x,self.Y:train_y,
                                     self.keep_rate:0.7})
    
            
            #self.cost = tf.losses.softmax_cross_entropy(self.Y, self.H)
    
    def get_prediction(self,test_x, rate):
        H_val = self.sess.run(self.H, feed_dict={self.X:test_x,
                                                          self.keep_rate:1})
        return H_val
    def get_accuracy(self,test_x,test_y,rate):
        return self.sess.run(self.accuracy, feed_dict={self.X:test_x
                                                            ,self.Y:test_y,
                                                      self.keep_rate:rate})

#H sess run , x 값은 피드딕트로 넣고, 
# 하나의 데이터에 대해 각각의 모델을 돌려 각각의 값을 도출 세로로 더해서 Hsum 
# 더한 후 가장 큰 값의 인덱스 는 argmax가 찾아줌. 
#백 개의 데이터마다 앙상블 정확도를 구하고, 모두 합한 뒤, 전체 데이터개수로 나눠준다. 

#1. 데이터 로딩
data = pd.read_csv("./Data/Digit/train.csv", sep = ",")
test = pd.read_csv("./Data/Digit/test.csv", sep=",")
test = np.array(test.values, dtype = np.float32)

sess = tf.Session()
seg = int(data.shape[0]*0.7)
train_data = data.loc[:seg,:]
test_data = data.loc[seg:,:]

train_x = np.array(train_data.iloc[:,1:].values)
train_y = sess.run(tf.one_hot(train_data.iloc[:,0],10))

test_x = np.array(test_data.iloc[:,1:].values)
test_y = sess.run(tf.one_hot(test_data.iloc[:,0],10))



# 2. 모델 객체를 생성
num_of_epoch = 1
batch_size = 100
num_of_models =2                                                            
for i in range(num_of_models):
    model = CNNModel(("model"+ str(i)), train_x,train_y,sess)
    model.build_model()
    sess.run(tf.global_variables_initializer())
    for step in range(num_of_epoch):
        num_of_iter = int(model.x.shape[0]/batch_size)
        for i in range(num_of_iter): #for 문은 아래로 내려야 한다. 
            batch_x = model.x[i*(batch_size):i*(batch_size)+(batch_size)]
            batch_y = model.y[i*(batch_size):i*(batch_size)+(batch_size)]
#             idx = idx + batch_size
            model.sess.run([model.train, model.cost], feed_dict = {model.X: batch_x, model.Y: batch_y,model.keep_rate:1 })
    model.get_prediction(test_x, 1) 
print("Model {} 정확도: {}".format((i,model.get_accuracy(test_x, test_y, 1))))

    
### 3. 결과 도출

# num_of_iter = int(test.shape[0] / batch_size)
# result = []
# idx = 0  
# for i in range(num_of_iter):
#     batch_x = test[idx:idx+batch_size,:]
#     batch_y = test_y[idx:idx+batch_size,:]
#     idx = idx + batch_size
     
#     prediction = np.zeros([batch_size,10])
    
#     for model_idx in range(num_of_models):
#         p = models[model_idx].get_prediction(batch_x,1)
#         prediction += p
# #     ## prediction을 구했어요
    
#     l_predict = tf.argmax(prediction,1)
#     tmp = sess.run(l_predict)
#     result.extend(tmp)

# df1 = pd.DataFrame([x+1 for x in range(test.shape[0])], columns=["ImageId"])
# df2 = pd.DataFrame(result, columns=["Label"])
# df3 = df1.join(df2)
# df3.to_csv("./Data/Digit/submission.csv", index=False)

ValueError: Variable model1/conv2d/kernel already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "D:\Anaconda3\envs\data_env\lib\site-packages\tensorflow\python\framework\ops.py", line 1770, in __init__
    self._traceback = tf_stack.extract_stack()
  File "D:\Anaconda3\envs\data_env\lib\site-packages\tensorflow\python\framework\ops.py", line 3274, in create_op
    op_def=op_def)
  File "D:\Anaconda3\envs\data_env\lib\site-packages\tensorflow\python\util\deprecation.py", line 488, in new_func
    return func(*args, **kwargs)
